In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
# Ensure you have the necessary YOLO weights and config files.
# Download YOLO weights: https://pjreddie.com/media/files/yolov3.weights
# Download YOLO config: https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg
# Download COCO names: https://github.com/pjreddie/darknet/blob/master/data/coco.names

# Path to YOLO config and weights files
yolo_cfg = 'yolov3.cfg'
yolo_weights = 'yolov3.weights'
yolo_names = 'coco.names'


In [ ]:
# Load YOLO model
net = cv2.dnn.readNet(yolo_weights, yolo_cfg)

# Load COCO names
with open(yolo_names, 'r') as f:
    classes = f.read().strip().split('\n')

In [ ]:
# Function to process an image and run YOLO detection
def detect_objects(image_path):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        draw_prediction(image, class_ids[i], confidences[i], x, y, x + w, y + h)

    return image

In [ ]:
# Helper functions
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

def draw_prediction(image, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    if label == 'car':  # Only draw bounding box for cars
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(image, (x, y), (x_plus_w, y_plus_h), color, 2)
        cv2.putText(image, f'{label} {confidence:.2f}', (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


In [ ]:
# Process training images (example)
train_dir = 'training_images'
for img_file in os.listdir(train_dir):
    img_path = os.path.join(train_dir, img_file)
    detected_image = detect_objects(img_path)
    cv2.imshow('Detected Image', detected_image)
    cv2.waitKey(0)

cv2.destroyAllWindows()